# DSC 10 Discussion 07: Permutation Tests, Bootstapping & Confidence Intervals

<img src="data/panda_tree.jpg" width="500">

#### Extra
- You can find additional help on [Permutation Tests](https://notes.dsc10.com/05-hypothesis_testing/2_permutation_tests.html) and [Bootstrapping Basics](https://notes.dsc10.com/06-estimation/1_bootstrap.html) in the Course Notes.
- [Here](https://drive.google.com/file/d/1mQApk9Ovdi-QVqMgnNcq5dZcWucUKoG-/view) is a pointer to that reference sheet we saw last time.

# Permutation Testing (Review)

###  A/B testing through simulation
- Decide whether two random samples come from the same distribution
- Statistic : difference between means
- Null hypothesis : the two groups are sampled from the same distribution
    - *PROBLEM* : we don't know what that distribution is!
    
### Permutation tests
- We can't draw samples from a distribution like we're used to because we don't know what the distribution is!
- Instead : randomly shuffle (permute) group labels during simulation
    - Compute the "difference in means" test statistic between groups of shuffled data

### Causation
- Observational study - rejecting the null hypothesis does not establish causality 
    - Correlation â‰  causation
    - Confounding factors
- Randomized Controlled Trial (RCT)
    - A/B test in a RCT does support causality

In [1]:
import babypandas as bpd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import otter
grader = otter.Notebook()

from notebook.services.config import ConfigManager

cm = ConfigManager()
cm.update(
    "livereveal", {
        'width': 1500,
        'height': 700,
        "scroll": True,
})

## Life expectancy data

This data comes from the World Health Organization.  We can learn more about the meanings of the columns by looking here: https://www.kaggle.com/kumarajarshi/life-expectancy-who

Let's travel back in time to the year 2015 and collect some data!  For the duration of this discussion, we're going to consider the following data to be our **"population"**.

Let's take a look at it.

In [2]:
# load in all the data
life_expectancy = bpd.read_csv("data/Life Expectancy Data.csv")
life_expectancy

In [3]:
# choose only data from 2015
recent_data = life_expectancy[life_expectancy.get("Year") == 2015]
recent_data

## Sampling from the data

From now on, the above data will be considered our population, so we will sample from this population to complete the following experiment.

First let us therefore take a sample of 50 countries from the population.

In [4]:
# grab a sample
recent_sample = recent_data.sample(50,replace=False).get(["Status","Life expectancy"])
recent_sample

---

## (Exercise) Section 1: Life expectancy and country status

Question : **Is life expectancy of people born in developing countries significantly shorter than that of people born in developed countries?**

### Question 1.1

Set up the null and alterantive hypotheses for this experiment.

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q1_01
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Question 1.2 

How many countries in each group?

In [5]:
countries_per_group = ...
countries_per_group

In [ ]:
grader.check("q1_02")

### Question 1.3

What is the average life expectancy in each group?

In [7]:
expectancy_per_group = ...
expectancy_per_group

In [ ]:
grader.check("q1_03")

### Question 1.4

Visualize the distribution of life expectancy for each group in a histogram plot.

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q1_04
manual: true
-->

In [9]:
# create a new dataframe for each group
developed_expectancy = ...
developing_expectancy = ...

# check to make sure the counts match those from above
print(f"There are {developed_expectancy.shape[0]} developed countries and {developing_expectancy.shape[0]} developing countries")

<!-- END QUESTION -->

In [10]:
# plot on a histogram
fig, ax = plt.subplots()

# first plot developed 
...

# then plot developing
...

plt.legend(["Developed", "Developing"])
graph = plt.xlabel("Life expectancy")

### Question 1.5

What test statistic should we use to compare these two sample distributions? 
Decide on which test statistic is best then compute it for the observered sample.

Hint (use ```expectancy_per_group``` from above)

In [11]:
# observed test statistic
test_statistic_name = ...

observed_test_statistic = ...
observed_test_statistic

In [ ]:
grader.check("q1_05")

### Question 1.6

Randomly permute the life expectancy (or the group labels) and create a new dataframe based on ```recent_sample``` with an additional column.

In [13]:
original_and_shuffled = recent_sample.assign(
    shuffled_life_expectancy = ...
)
original_and_shuffled

In [ ]:
grader.check("q1_06")

### Question 1.7

Compute the mean life expectancy for each group in the newly permuted data.

What do you notice?

In [16]:
diff_between_means = ...
diff_between_means

In [ ]:
grader.check("q1_07")

### Question 1.8

Is it not clear? Let's try taking the mean difference

In [18]:
obs_mean_difference = ...
obs_mean_difference

### Question 1.9
Wow! That's a huge difference? Could it be chance? Let's repeat this 5000 times and store the shuffled difference in an array

In [19]:
simulated_stats = np.array([]) # BEGIN SOLUTION
num_observations = 5000 # BEGIN SOLUTION

...
simulated_stats
    

In [ ]:
grader.check("q1_09")

### Question 1.10

Doesn't look like we can blame these differences on chance (assuming our null hypothesis is true). Let's see the likelihood of our observed difference given this result.

In [21]:
p_val = ...
p_val

In [ ]:
grader.check("q1_10")

In [23]:
bpd.DataFrame().assign(DifferenceInMeans=simulated_stats).plot(kind='hist', bins=9, density=True, ec='w', figsize=(10, 5));
plt.axvline(observed_test_statistic, color='red', label='observed difference')
plt.legend();

### Question 1.11

Can we say that life expectancy of people born in developing countries is shorter than that of people born in developing countries?

In [24]:
reject_null_hypothesis = ...
reject_null_hypothesis

In [ ]:
grader.check("q1_11")

### Question 1.12

Can we conclude that lower/higher life expectancy directly causes the nation to be more/less developed or vice-versa?

In [26]:
conclude_causation = ...
conclude_causation

In [ ]:
grader.check("q1_12")

No, you cannot determine causation since this is observational data. There might be lot of other factors that can link life expectancy to nation's development.

---

# Bootstrapping - sampling within a sample
- Problem : statistics about the data population are often unavailable, costly to acquire, unknown, etc.
- Solution : utilize random sampling (and re-sampling) of available data to estimate population statistics
    - The result of bootstrapping will be a distribution over sample statistics!
    - Hopefully we'll see that these *sample statistics* $\approx$ *population statistics*
    
### Bootstrapping basic procedure
- Sample from the population
- Re-sample from that same sample (make sure to have replace=True!)
- Repeat
- **Note** - after re-sampling, we will likely see duplicate data entries within a single sample, but that's okay! 
    - If we didn't have duplicates, then we would have the same exact data in every single sample (this would be bad!)

# Bootstrapped Confidence Intervals
- Goal : return a range of values that we are confident contain the true population statistic 
    - Bootstrapping gives us a distribution of sample statistics
    - The true population statistic often lies within the bulk of that distribution 
- $X$% confidence interval 
    - Interpretation
        - **YES**: $X$% of all bootstrapped sample statistics fall within that interval
        - **YES**: ~$X$% of the time, the interval will capture the correct population statistic
        - **YES**: I'm $X$% sure that the true population statistic is in the interval
        - **NO**: the true population statistic has an $X$% change of being in the interval
    - Computation
        - Use $\frac{100-X}{2}$ and $100-\frac{100-X}{2}$ for lower and upper percentiles
        
### CIs for testing
- Given P-value $p$ and null hypothesis "population statistic = $a$":
    - Construct $(100-p)$ CI for populatiton statistic
    - Reject null hypothesis if $a$ is not in the interval

# Describing a Distribution : Mean and Spread
- Center of a distribution 
    - *Mean* : balance point
    - *Median* : half-way point (robust to outliers) 
- Spread of distribution 
    - *Range* : biggest - smallest
    - *Standard deviation* : variability around the mean
- Chebyshev's Inequality
    - Proportion of values in the range "average $\pm\ z$ SDs" is â‰¥ $1-\frac{1}{z^2}$
- Looking forward
    - We'll look at other types of distributions and ones that can be parameterized 

# Section 2: Sampling and bootstrapping

Here we'll take a look at the same life expectancy data and do some sampling exercises.

<div style="padding: 15px; border: 1px solid transparent; border-color: transparent; margin-bottom: 20px; border-radius: 4px; color: #3c763d; background-color: #dff0d8; border-color: #d6e9c6;">
  
Quick outline
- data : life expectancy 
- population : all countries
- sample : smaller random selection of countries
- visualizations : histograms of life expectancy data and mean
    
</div>

In [28]:
recent_data

In [29]:
# Let's visualize our population distribution.

# Defining a function to create bins easily
def get_bins(array, bin_size=1):
    smallestNum = int(array.min())
    
    largestNum = int(array.max())
    upperLimit = largestNum + bin_size + 1
    
    return np.arange(smallestNum, upperLimit, bin_size)

In [30]:
measured = recent_data.get("Life expectancy")

#generate number of bins
n_bins = get_bins(measured, 1) # <-- Try playing around with the bin size

#lets plot the histogram
recent_data.get('Life expectancy').plot(kind='hist', bins=n_bins, density=True)

In [31]:
# Let's check the population mean
pop_mean = recent_data.get('Life expectancy').mean()
pop_mean

### Question 2.1
Is this the population distribution or sample distribution?

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q2_01
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## So, what is our aim?  

We want to estimate the average life expectancy for the globe!  Let's say we don't have access to the entire population (which is the case usually)

Flying around the world is pretty expensive, so we can only collect data from 15 countries.

We can sampling and use bootstrapping to find this.


In [32]:
# different sample sizes

num_samples = 40

# num_samples = 50

In [33]:
# How do we create a representative sample?
collected = recent_data.sample(n=num_samples, replace=False)
collected

In [34]:
#we need new bin sizes
n_bins = get_bins(collected.get('Life expectancy'),1)

#lets plot the histogram
plt.title("Sample Distribution")
collected.get('Life expectancy').plot(kind='hist', bins=n_bins, density=True)
plt.show()

### Question 2.2

Is this the population distribution or sample distribution?

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q2_02
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



We're interested in estimating the mean life expectancy.  So, let's find the mean of our sample.

In [35]:
sample_mean = collected.get('Life expectancy').mean()
sample_mean

In [36]:
# We can show our mean in relation to the sample:

#plot the historgram again
collected.get('Life expectancy').plot(kind='hist', bins=n_bins, density=True)

#draw the sample mean
plt.title("Sample Mean")
plt.axvline(sample_mean, c='r')
plt.show()

### Question 2.3

Is this sample mean or population mean? Does this change by the taking a new sample?

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q2_03
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Resampling

What happens when we resample?

**Resampling means not taking a new sample, but sampling from the single sample with replacement.** We do this since getting different samples is difficult in real world scenarios.

In [37]:
# Run this multiple time to see what changes.

resampled = collected.sample(num_samples,replace=True)
resampled_mean = resampled.get('Life expectancy').mean()
n_bins = get_bins(collected.get('Life expectancy'), 1)

print("The resampled mean is:\t\t", resampled_mean, "\nCompared to the original sample:", sample_mean)

#plot the historgram again
resampled.get('Life expectancy').plot(kind='hist', bins=n_bins, density=True)

#lets show the sampled_mean and resampled_mean
plt.title("Resampled mean")
plt.axvline(resampled_mean, c='r')
plt.axvline(sample_mean, c='b')
plt.show()

### Question 2.4

Is this a sample mean or resampled mean?

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q2_04
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



Now, let's run the bootstrap so we can create a distribution!

In [38]:
sample_means = np.array([])

for i in range(5000):
    bootstrapped = collected.sample(num_samples,replace=True)
    boot_mean = bootstrapped.get('Life expectancy').mean()
    sample_means = np.append(sample_means, boot_mean)
    

plt.title("Distribution of Sample Means")
plt.hist(sample_means, bins=get_bins(sample_means, 0.5))
plt.show()

### Question 2.5 

What is this distribution?

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q2_05
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Compare with the popultation mean too!
- comparing population mean to the distribution of sample means

In [39]:
plt.hist(sample_means, bins=get_bins(sample_means, 0.5))
plt.scatter(pop_mean, 0, color='red', s=80).set_zorder(2)

# So now what? 

- What conclusions can we make about the **population mean** based on our distribution of sample means?

---

# Section 3: Confidence Intervals

- We would like to come up with a range of values that contain X% of all bootstrapped sample means. 
- This interval corresponds to an X% confidence interval

### How to do this?
- We need our array of sample means and we need to compute a few percentiles based on what X% confidence interval we'd like to return

### Question 3.1 
Suppose we'd like to construct 90% and 82% Confidence Intervals over some statistic.

What are the upper and lower percentiles we need in each case?

In [40]:
# compute the lower percentile given a confidence interval
def compute_lower_percentile(perc_conf):
    
    lower_perc = ...
    
    return lower_perc

# compute the upper percentile given a confidence interval
def compute_upper_percentile(perc_conf):
    
    upper_perc = ...
    
    return upper_perc

In [41]:
lower_perc_90 = ...
print(f"Lower percentile for 90% C.I. : {lower_perc_90}")

upper_perc_90 = ...
print(f"Upper percentile for 90% C.I. : {upper_perc_90}")

In [42]:
lower_perc_82 = ...
print(f"Lower percentile for 82% C.I. : {lower_perc_82}")

upper_perc_82 = ...
print(f"Upper percentile for 82% C.I. : {upper_perc_82}")

### Question 3.2 

Which of the two confidence intervals (90% or 82%) is larger? Why?

In [43]:
# choose 90 or 82
 
larger_ci = ...

...

### Question 3.3

Compute the upper and lower bounds of a 95% confidence interval for our ```sample_means``` data from above.

In [44]:
def compute_ci(confidence_level, sample_means):

    # What is the mean we're estimating?
    mean = ...

    # What are the percentiles?
    # Use the functions we made above
    lower_perc = ...
    upper_perc = ...

    # And then our lower and upper bounds?
    lower_bound = ...
    upper_bound = ...

    # Printing it out so we can easily see our results.
    print("""
    Mean:\t{}

    Lower Percentile:\t{}
    Upper Percentile:\t{}

    Lower Bound:\t{}
    Upper Bound:\t{}

    Confidence Level:\t{}%
    """.format(mean, lower_perc, upper_perc, lower_bound, upper_bound, confidence_level))
    
    return lower_bound, upper_bound

In [45]:
confidence_level = ...

# compute the ci
...

### Lets visualize the confidence interval on the histogram from earlier

In [46]:
def plot_ci(ci, lower_bound, upper_bound, sample_means, pop_mean):
    plt.title(f"{ci}% confidence interval")
    plt.hist(sample_means, bins=get_bins(sample_means, 0.5))
    plt.scatter(pop_mean, 0, color='red', s=80).set_zorder(3)
    plt.plot([lower_bound, upper_bound], [0,0], color='lime', linewidth=4, zorder=2)

In [47]:
plot_ci(confidence_level, lower_bound, upper_bound, sample_means, pop_mean)

### Question 3.4
Interpret what the confidence interval means in the context of this problem. 

In [48]:
interpretation = ...

In [49]:
lower_bound

In [50]:
upper_bound

### Question 3.5

Compute 100%, 80%, and 50% confidence intervals using the same ```sample_means``` and visualize the results of each.

In [51]:
# compute the bounds
print("100% CI")
...

print("80% CI")
...

print("50% CI")
...

In [52]:
# visualize the results
plot_ci(100, lower_100, upper_100, sample_means, pop_mean)
plot_ci(80, lower_80, upper_80, sample_means, pop_mean)
plot_ci(50, lower_50, upper_50, sample_means, pop_mean)

### Question 3.6

Do any of the above confidence intervals (100%, 95%, 80%, 50%) NOT contain the true population mean?

In [53]:
pop_mean

In [54]:
# answer True or False
exists_interval = ...
exists_interval

### Question 3.7

Is it possible for the 80% confidence interval to contain the true population mean while the 95% confidence interval does not?

In [55]:
# answer True or False
possible = ...
possible